In [77]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score 
import pickle
import joblib

df=pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv')
df.head()

,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr
0,1,50002,1,50002,PITT,no_filename,1,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452
1,2,50003,2,50003,PITT,Pitt_0050003,1,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155
2,3,50004,3,50004,PITT,Pitt_0050004,1,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463
3,4,50005,4,50005,PITT,Pitt_0050005,1,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200
4,5,50006,5,50006,PITT,Pitt_0050006,1,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574


In [78]:
df.describe()
print('Shape of initial dataset:')
print(df.shape)

#Checking for null values to fill
print("Number of null values:")
print(df.isnull().sum())

Shape of initial dataset:
(1112, 13)
Number of null values:
Unnamed: 0     0
SUB_ID         0
X              0
subject        0
SITE_ID        0
FILE_ID        0
DX_GROUP       0
anat_cnr      13
anat_efc      13
anat_fber     13
anat_fwhm     13
anat_qi1      13
anat_snr      13
dtype: int64


In [79]:
#Dropping empty columns
df['anat_cnr'].replace('', np.nan, inplace=True)
df['anat_efc'].replace('', np.nan, inplace=True)
df['anat_fber'].replace('', np.nan, inplace=True)
df['anat_fwhm'].replace('', np.nan, inplace=True)
df['anat_qi1'].replace('', np.nan, inplace=True)
df['anat_snr'].replace('', np.nan, inplace=True)

#Replacing null values in all relevant input columns
df.dropna(subset=['anat_cnr','anat_efc', 'anat_fber', 'anat_fwhm', 'anat_qi1', 'anat_snr'], inplace=True)

#Verifying number of null rows
print("Number of null values:")
print(df.isnull().sum())

Number of null values:
Unnamed: 0    0
SUB_ID        0
X             0
subject       0
SITE_ID       0
FILE_ID       0
DX_GROUP      0
anat_cnr      0
anat_efc      0
anat_fber     0
anat_fwhm     0
anat_qi1      0
anat_snr      0
dtype: int64


In [80]:
#Checking number of unique values and wrong entries like symbols -,?,#,*,etc.
for col in df.columns:
    print('{} : {}'.format(col,df[col].unique()))

Unnamed: 0 : [   1    2    3 ... 1110 1111 1112]
SUB_ID : [50002 50003 50004 ... 51585 51606 51607]
X : [   1    2    3 ... 1110 1111 1112]
subject : [50002 50003 50004 ... 51585 51606 51607]
SITE_ID : ['PITT' 'OLIN' 'OHSU' 'SDSU' 'TRINITY' 'UM_1' 'UM_2' 'USM' 'YALE' 'CMU'
 'LEUVEN_1' 'LEUVEN_2' 'KKI' 'NYU' 'STANFORD' 'UCLA_1' 'UCLA_2' 'MAX_MUN'
 'CALTECH' 'SBL']
FILE_ID : ['no_filename' 'Pitt_0050003' 'Pitt_0050004' ... 'SBL_0051585'
 'MaxMun_a_0051606' 'MaxMun_a_0051607']
DX_GROUP : [1 2]
anat_cnr : [10.20153877  7.16570147  7.69814438 ...  3.41346894  7.83900677
 12.16929687]
anat_efc : [1.19466382 1.12675161 1.22621772 ... 1.35823764 1.75436261 2.81835195]
anat_fber : [16.22345825 10.4600076   9.72575046 ...  4.33569983 12.27005481
  9.27210712]
anat_fwhm : [3.8780004  4.28223801 3.88168429 ... 3.32455    3.23217    3.51019   ]
anat_qi1 : [0.15271098 0.16171559 0.17418572 ... 0.10948991 0.08396437 0.04430978]
anat_snr : [12.07245188  9.24115456  9.32346277 ...  4.93395956 16.403173

In [81]:
np.isnan(df.values.any()) 

False

In [82]:
# Split dataset into training set and test set
X=df[['anat_cnr','anat_efc', 'anat_fber', 'anat_fwhm', 'anat_qi1', 'anat_snr']]
y=df['DX_GROUP']
X_train,X_test,y_train,y_test=train_test_split(X.values,y,test_size = 0.3, random_state = 1000)

#Create model
#RandomForestClassifier(n_estimators=100)
model=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

In [83]:
from sklearn import metrics

#Classification accuracy
print(metrics.classification_report(y_test, model.predict(X_test)))

print(accuracy_score(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           1       0.51      0.40      0.45       160
           2       0.53      0.64      0.58       170

    accuracy                           0.52       330
   macro avg       0.52      0.52      0.51       330
weighted avg       0.52      0.52      0.51       330

0.5212121212121212
